In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import col, to_date, date_format, sum as spark_sum

In [3]:
sc = SparkSession.builder.appName("test-spark").getOrCreate()

24/06/14 11:24:56 WARN Utils: Your hostname, khoi-ROG-Strix-G512LI-G512LI resolves to a loopback address: 127.0.1.1; using 192.168.1.239 instead (on interface wlo1)
24/06/14 11:24:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/14 11:24:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df1 = sc.read.parquet("hdfs:///log_action_test/88508dea-97e5-400f-b25d-210c32b85468")

In [5]:
df1.show()

+------------+--------+---------+---------+
|student_code|activity|numOfFile|timestamp|
+------------+--------+---------+---------+
|          33|    read|        5|6/12/2024|
|          33| execute|        1|6/13/2024|
|           6|   write|        6|6/15/2024|
|          24| execute|        8|6/12/2024|
|          21|    read|        5|6/12/2024|
|          24|    read|       10|6/12/2024|
|           3|    read|        9|6/13/2024|
|          21|   write|        9|6/12/2024|
|          21|    read|        1|6/15/2024|
|          24|   write|        5|6/15/2024|
|          24| execute|       10|6/10/2024|
|          21| execute|        6|6/15/2024|
|          27|   write|        7|6/14/2024|
|          39|   write|        1|6/10/2024|
|          27| execute|       10|6/12/2024|
|          15|   write|        3|6/13/2024|
|          27|   write|        1|6/10/2024|
|          39| execute|        5|6/13/2024|
|          30|   write|        6|6/10/2024|
|          18|    read|        4

In [6]:
print(type(df1))

<class 'pyspark.sql.dataframe.DataFrame'>


In [7]:
activity_df = df1.withColumn("timestamp", date_format(to_date(col("timestamp"), "M/d/yyyy"), "yyyyMMdd"))

In [8]:
students_info_df = sc.read.option("header", "false").csv("/students_info_path/danh_sach_sv_de.csv")

In [9]:
columns = ["student_code", "student_name"]
students_info_df = students_info_df.toDF(*columns)

In [10]:
students_info_df.show()

+------------+--------------------+
|student_code|        student_name|
+------------+--------------------+
|           1|          Mai Đức An|
|           2|      Nguyễn Mai Anh|
|           3|   Ngô Ngọc Tuấn Anh|
|           4|      Trần Trung Anh|
|           5|       Trần Ngọc Bảo|
|           6|  Nguyễn Vũ Hòa Bình|
|           7|    Nguyễn Thành Đạt|
|           8|        Đỗ Thành Đạt|
|           9|    Nguyễn Khoa Đoàn|
|          10|    Nguyễn Quốc Dũng|
|          11|     Đường Minh Quân|
|          12|   Dương Quang Giang|
|          13|    Nguyễn Minh Hiếu|
|          14|        Ngô Phi Hùng|
|          15|Nguyễn Đình Thiên...|
|          16|        Đỗ Doãn Khắc|
|          17|      Châu Minh Khải|
|          18|      Phạm Đình Khôi|
|          19|        Lê Bảo Khánh|
|          20|        Lê Minh Phúc|
+------------+--------------------+
only showing top 20 rows



In [11]:
joined_df = activity_df.join(students_info_df, "student_code","outer")

In [12]:
joined_df.show()

+------------+--------+---------+---------+-----------------+
|student_code|activity|numOfFile|timestamp|     student_name|
+------------+--------+---------+---------+-----------------+
|           1|    read|        5| 20240613|       Mai Đức An|
|           1|    read|        8| 20240610|       Mai Đức An|
|           1|    read|        9| 20240613|       Mai Đức An|
|           1|    read|        5| 20240611|       Mai Đức An|
|           1|    read|        6| 20240611|       Mai Đức An|
|           1|    read|        9| 20240610|       Mai Đức An|
|           1|   write|        4| 20240614|       Mai Đức An|
|           1| execute|        3| 20240611|       Mai Đức An|
|           1| execute|       10| 20240612|       Mai Đức An|
|           1|    read|        8| 20240610|       Mai Đức An|
|           1|   write|        6| 20240610|       Mai Đức An|
|           1|   write|       10| 20240613|       Mai Đức An|
|           1|    read|        7| 20240615|       Mai Đức An|
|       

In [13]:
result_df = joined_df.groupBy("timestamp", "student_code", "student_name", "activity") \
    .agg(spark_sum("numOfFile").alias("totalFile"))

In [14]:
result_df.show()

+---------+------------+--------------------+--------+---------+
|timestamp|student_code|        student_name|activity|totalFile|
+---------+------------+--------------------+--------+---------+
| 20240614|           1|          Mai Đức An|   write|        4|
| 20240615|          19|        Lê Bảo Khánh| execute|       15|
| 20240612|           3|   Ngô Ngọc Tuấn Anh|   write|        2|
| 20240610|           5|       Trần Ngọc Bảo| execute|        5|
| 20240612|          17|      Châu Minh Khải|    read|       15|
| 20240614|          36|        Vũ Khắc Long|    read|        5|
| 20240610|          27|Nguyễn Thị Thùy T...|   write|        1|
| 20240612|          24|     Nguyễn Hoài Nam| execute|       18|
| 20240610|          29|    Nguyễn Minh Quân|   write|        9|
| 20240611|           4|      Trần Trung Anh| execute|        7|
| 20240615|          19|        Lê Bảo Khánh|   write|        4|
| 20240612|          25|    Đào Thanh Nguyên| execute|        9|
| 20240610|          30| 

In [15]:
sorted_df = result_df.orderBy("timestamp", "student_code", "activity")

In [16]:
sorted_df.show()

+---------+------------+--------------------+--------+---------+
|timestamp|student_code|        student_name|activity|totalFile|
+---------+------------+--------------------+--------+---------+
| 20240610|           1|          Mai Đức An|    read|       25|
| 20240610|           1|          Mai Đức An|   write|        6|
| 20240610|          10|    Nguyễn Quốc Dũng|   write|        5|
| 20240610|          11|     Đường Minh Quân| execute|        4|
| 20240610|          11|     Đường Minh Quân|   write|        5|
| 20240610|          12|   Dương Quang Giang|    read|        8|
| 20240610|          13|    Nguyễn Minh Hiếu|    read|        8|
| 20240610|          15|Nguyễn Đình Thiên...|    read|        1|
| 20240610|          15|Nguyễn Đình Thiên...|   write|        8|
| 20240610|          16|        Đỗ Doãn Khắc|    read|        6|
| 20240610|          16|        Đỗ Doãn Khắc|   write|        1|
| 20240610|          17|      Châu Minh Khải| execute|       14|
| 20240610|          17| 

In [17]:
sorted_df.write.option("header", "true").csv("/ouput_student_path")